# Лабораторная работа №2

**Задание выполнил Артамонов Кирилл, гр. Б01-007.**

Импортируем библиотеки:

In [1]:
import numpy as np
import matplotlib.pyplot as plt


Зададим систему **(пункт У)** Ax = f :

In [28]:
n = 12 # matrix size
EPSILON = 1e-15 # Accuracy

matrixA = np.zeros((n, n))

for i in range(0, n):
    for j in range(0, n):
        if i == j:
            matrixA[i][j] = 1
        else:
            matrixA[i][j] = 1 / ((i+1)*(i+1) + j + 1)

vectorF = np.array([1 / (i + 1) for i in range(0, n)])


In [29]:
def getDiscrepancy(mat, x, F):
    return np.linalg.norm(F - np.matmul(mat, x))

def getMaxEigenvalue(mat):
    v1 = np.array(np.random.rand(n))
    v2 = np.matmul(mat, v1)
    while getDiscrepancy(mat, v1, v2) > EPSILON:
        v2 = np.matmul(mat, v1)
    eigMax = np.linalg.norm(np.matmul(mat, v1)) / np.linalg.norm(v1)
    return eigMax

def getEigenvalues(mat):
    return 1 / getMaxEigenvalue(np.linalg.inv(mat)), getMaxEigenvalue(mat)

def getConditionNumber(mat):
    eigMin, eigMax = getEigenvalues(mat)
    return np.abs(eigMax / eigMin)

**LU**-разложение

In [30]:
def checkLU(A):
    for i in range(0, n):
        mat_i = np.array([lines[0: i + 1] for lines in A[0: i + 1]])
        if not np.linalg.det(mat_i):
            return False
    return True

def decomposeLU(A):
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    for i in range(0, n):
        L[i][i] = 1

    for i in range(0, n):
        for j in range(0, n):
            sum = 0
            if i <= j:
                for k in range(0, i):
                    sum += L[i][k] * U[k][j]
                U[i][j] = A[i][j] - sum
            if i > j:
                for k in range(0, j):
                    sum += L[i][k] * U[k][j]
                L[i][j] = (A[i][j] - sum) / U[j][j]

    return L, U

def _selectElement(A, F, j, range): #service function for Gauss method
    for i in range:
        if A[i][j] > A[j][j]:
            A[i], A[j] = A[j], A[i]
            F[i], F[j] = F[j], F[i]

def _runGauss(A, F, j, range): #service function for Gauss method
    for i in range:
        F[i] -= F[j] * (A[i][j] / A[j][j])
        A[i] -= A[j] * (A[i][j] / A[j][j])
    F[j] /= A[j][j]
    A[j] /= A[j][j]

def _directCourse(A, F):
    for j in range(0, n):
        _selectElement(A, F, j, range(j + 1, n))
        _runGauss(A, F, j, range(j + 1, n))

def _reverseCourse(A, F):
    for j in reversed(range(0, n)):
        _selectElement(A, F, j, reversed(range(0, j)))
        _runGauss(A, F, j, reversed(range(0, j)))

def GaussMethod(A, F):
    copyA = A.copy()
    copyF = F.copy()

    _directCourse(copyA, copyF)
    _reverseCourse(copyA, copyF)
    return copyF

def LUMethod(A, F):
    if not checkLU(A):
        return GaussMethod(A, F)

    copyF = F.copy()
    L, U = decomposeLU(A)
    _directCourse(L, copyF)
    _reverseCourse(U, copyF)    
    return copyF


Метод верхней релаксации:

In [31]:
omega = 1.5

def UpperRelaxationMethod(A, F):
    copyA = A.copy()
    copyF = F.copy()

    L = np.zeros((n, n))
    D = np.zeros((n, n))
    U = np.zeros((n, n))

    for i in range(0, n):
        D[i][i] = A[i][i]
        for j in range(i + 1, n):
            U[i][j] = A[i][j]
            L[j][i] = A[j][i]

    inv = np.linalg.inv(D + omega * L)
    B = - np.matmul(inv, (omega - 1) * D + omega * U)
    f = np.matmul(omega * inv, F)

    x = np.array(np.random.rand(n))

    while getDiscrepancy(copyA, x, copyF) > EPSILON:
        x = np.matmul(B, x) + f

    return x

Результаты:

In [33]:
eigMin, eigMax = getEigenvalues(matrixA)
print("min|λ(A)| =", eigMin)
print("max|λ(A)| =", eigMax)
print("Число обусловленности:", getConditionNumber(matrixA))
print("Критерий останова: |Ax - f| < ε =", EPSILON)
print('\n=======================================================================\n')
print("Метод LU-разложения:\n")
x = LUMethod(matrixA, vectorF)
print("Корень системы:\n", x)
print("Невязка метода: ", getDiscrepancy(matrixA, x, vectorF))
print('\n=======================================================================\n')
print("Метод верхней релаксации:\n")
y = UpperRelaxationMethod(matrixA, vectorF)
print("Корень системы:\n", y)
print("Невязка метода: ", getDiscrepancy(matrixA, y, vectorF))
print('\n=======================================================================\n')

min|λ(A)| = 1.172658512931444
max|λ(A)| = 1.4136857502390958
Число обусловленности: 1.206497263530192
Критерий останова: |Ax - f| < ε = 1e-15


Метод LU-разложения:

Корень системы:
 [0.75537042 0.23879963 0.17890325 0.15086988 0.13170938 0.1170332
 0.10527117 0.09560544 0.08752166 0.08066593 0.074783   0.06968322]
Невязка метода:  2.443436683771534e-16


Метод верхней релаксации:

Корень системы:
 [0.75537042 0.23879963 0.17890325 0.15086988 0.13170938 0.1170332
 0.10527117 0.09560544 0.08752166 0.08066593 0.074783   0.06968322]
Невязка метода:  5.825365706983819e-16


